In [ ]:
import imblearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, recall_score, make_scorer

In [ ]:
# Generate a synthetic imbalanced dataset with 300 samples and 12 features
X, y = make_classification(n_samples=300, n_features=12 , random_state=5, weights=[260/298,38/298])

In [ ]:
# Define specificity and sensitivity scoring functions
def specificity_score(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label=0)
def sensitivity_score(y_true, y_pred):
    return recall_score(y_true, y_pred)

# Create scorers using make_scorer
specificity = make_scorer(specificity_score)
sensitivity = make_scorer(sensitivity_score)

# Define parameters for grid search
gamma_range = np.logspace(start=-5, stop=5, num=11, base=2)
C_range = np.logspace(start=-5, stop=5, num=11, base=10)
parameters_incor = {'C': C_range, 'gamma': gamma_range}

# Define scoring metrics for grid search including accuracy, sensitivity, and specificity
scoring = {'accuracy':'accuracy','sensitivity':sensitivity,'specificity':specificity}

In [ ]:
# Oversample the dataset using ADASYN
oversampler = ADASYN(n_neighbors=5, random_state=15)
X_oversamp,y_oversamp = oversampler.fit_resample(X, y)

# Shuffle ADASYN-generated samples to ensure random distribution across folds
X_oversamp,y_oversamp = shuffle(X_oversamp, y_oversamp, random_state=15)

# Initialize SVM model
svc_incor = SVC(kernel='rbf', random_state=5)

# Define GridSearchCV with custom scorers
clf_incor = GridSearchCV(svc_incor, parameters_incor, cv=10, scoring=scoring, refit='accuracy')

# Perform grid search
clf_incor.fit(X_oversamp, y_oversamp)

# Print results
print("Accuracy:", clf_incor.best_score_)
print("Error:", (1-clf_incor.best_score_))
print("Sensitivity:", clf_incor.cv_results_['mean_test_sensitivity'][clf_incor.best_index_])
print("Specificity:", clf_incor.cv_results_['mean_test_specificity'][clf_incor.best_index_])
print("Best hyperparameters:", clf_incor.best_params_)

In [ ]:
# Set the figure size
plt.figure(figsize=(10, 8))
# Reshape the mean test accuracy scores into a 2D array
scores = clf_incor.cv_results_["mean_test_accuracy"].reshape(len(C_range), len(gamma_range))
# Display the scores as a heatmap
plt.imshow(scores, interpolation="nearest", cmap=plt.cm.hot)
# Set the x-axis label
plt.xlabel("gamma")
# Set the y-axis label
plt.ylabel("C")
# Display the colorbar
plt.colorbar()
# Set the x-axis ticks and labels
plt.xticks(np.arange(gamma_range.shape[0]), labels=gamma_range, rotation=45)
# Set the y-axis ticks and labels
plt.yticks(np.arange(gamma_range.shape[0]), labels=C_range)
# Set the title of the plot
plt.title("Validation accuracy")
# Display the plot
plt.show()

In [ ]:
# Define the pipeline
model = imblearn.pipeline.Pipeline([
        ('ADASYN', imblearn.over_sampling.ADASYN(random_state=5, n_neighbors=5)),
        ('SVM', SVC(kernel='rbf', random_state=5))
    ])

# Define the parameters grid for GridSearchCV
parameters_cor={'SVM__C': C_range, 'SVM__gamma': gamma_range}
#'ADASYN__n_neighbors':[2,3,4,5]

# Define GridSearchCV with custom scorers
clf_cor = GridSearchCV(model, parameters_cor, cv=10, scoring=scoring, refit='accuracy')

# Perform grid search
clf_cor.fit(X, y)

# Print results
print("Accuracy:", clf_cor.best_score_)
print("Error:", (1-clf_cor.best_score_))
print("Sensitivity:", clf_cor.cv_results_['mean_test_sensitivity'][clf_cor.best_index_])
print("Specificity:", clf_cor.cv_results_['mean_test_specificity'][clf_cor.best_index_])
print("Best hyperparameters:", clf_cor.best_params_)

In [ ]:
# Set the figure size
plt.figure(figsize=(10, 8))
# Reshape the mean test accuracy scores into a 2D array
scores = clf_cor.cv_results_["mean_test_accuracy"].reshape(len(C_range), len(gamma_range))
# Display the scores as a heatmap
plt.imshow(scores, interpolation="nearest", cmap=plt.cm.hot)
# Set the x-axis label
plt.xlabel("gamma")
# Set the y-axis label
plt.ylabel("C")
# Display the colorbar
plt.colorbar()
# Set the x-axis ticks and labels
plt.xticks(np.arange(gamma_range.shape[0]), labels=gamma_range, rotation=45)
# Set the y-axis ticks and labels
plt.yticks(np.arange(gamma_range.shape[0]), labels=C_range)
# Set the title of the plot
plt.title("Validation accuracy")
# Display the plot
plt.show()